In [1]:
import pandas as pd
import numpy as np
import numpy as np
import os
from tensorflow.keras.applications import VGG19, InceptionV3
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions
from tensorflow.keras.utils import to_categorical
import numpy as np


In [2]:
path_test_ENDO = 'dados/endo'
path_test_OUTRO = 'dados/outros'

# Lista de todas as imagens
list_img_endo = os.listdir(path_test_ENDO)
list_img_outros = os.listdir(path_test_OUTRO)

x_data = []
y_data = []

# Lendo as imagens que tem endo
for img_name in list_img_endo:
    img_path = os.path.join(path_test_ENDO, img_name)
    img = image.load_img(img_path, target_size=(512, 512))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    x_data.append(img_array)
    y_data.append(1)  # Rótulo 0 para endo

# Lendo as imagens que tem outros
for img_name in list_img_outros:
    img_path = os.path.join(path_test_OUTRO, img_name)
    img = image.load_img(img_path, target_size=(512, 512))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(img_array)
    x_data.append(img_array)
    y_data.append(0)  # Rótulo 1 para outros

x_data = np.array(x_data)
y_data = np.array(y_data)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

print("Shape of x_train:", x_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of x_train: (92, 512, 512, 3)
Shape of x_test: (24, 512, 512, 3)
Shape of y_train: (92,)
Shape of y_test: (24,)


In [3]:
num_classes = 2
#VGG19 pré-treinado
#VGG19 sem as camadas totalmente conectadas
base_model = VGG19(weights='imagenet', include_top=False)

#camadas totalmente conectadas para a nova tarefa de classificação
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

#Novo modelo combinando a base pré-treinada com as camadas personalizadas
model = Model(inputs=base_model.input, outputs=predictions)

# Congelar as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-12-20 16:44:21.922911: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-20 16:44:21.924381: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
# Tranformando os rótulos em codificação one-hot
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

# Treinando o modelo
model.fit(x_train, y_train_one_hot, epochs=5, batch_size=2, validation_data=(x_test, y_test_one_hot))

# Medindo o desempenho do modelo nos dados de teste
accuracy = model.evaluate(x_test, y_test_one_hot)[1]
print(f'Acurácia nos dados de teste: {accuracy * 100:.2f}%')

Epoch 1/5


2023-12-20 16:44:25.034601: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-12-20 16:44:25.450534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


46/46 [==============================] - ETA: 0s - loss: 1.0972 - accuracy: 0.4674

2023-12-20 16:44:37.808140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


46/46 [==============================] - 16s 308ms/step - loss: 1.0972 - accuracy: 0.4674 - val_loss: 0.7321 - val_accuracy: 0.5833
Epoch 2/5
46/46 [==============================] - 14s 304ms/step - loss: 0.7670 - accuracy: 0.5109 - val_loss: 0.6540 - val_accuracy: 0.5833
Epoch 3/5
46/46 [==============================] - 14s 312ms/step - loss: 0.6541 - accuracy: 0.5870 - val_loss: 0.9485 - val_accuracy: 0.5833
Epoch 4/5
46/46 [==============================] - 15s 324ms/step - loss: 0.6002 - accuracy: 0.6848 - val_loss: 0.6224 - val_accuracy: 0.6250
Epoch 5/5
46/46 [==============================] - 15s 325ms/step - loss: 0.4981 - accuracy: 0.7391 - val_loss: 0.5930 - val_accuracy: 0.6250


2023-12-20 16:45:39.041931: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 7s 7s/step - loss: 0.5930 - accuracy: 0.6250
Acurácia nos dados de teste: 62.50%


In [5]:
model.save('test2_vgg19_0_62.h5')

In [6]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Medindo o desempenho do modelo nos dados de teste
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test_one_hot, axis=1)

# Métricas
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Matriz de confusão
conf_matrix = confusion_matrix(y_true, y_pred_classes)

print(f'Acurácia nos dados de teste: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('\nMatriz de Confusão:')
print(conf_matrix)


2023-12-20 16:45:58.252085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 678ms/step
Acurácia nos dados de teste: 62.50%
Precision: 0.6204
Recall: 0.6250
F1 Score: 0.6216

Matriz de Confusão:
[[ 5  5]
 [ 4 10]]
